# Space

In [ ]:
import os
import logging
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
KEY = 'WorkSpace'
WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY
# print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
import sys
from proj_space import SPACE
sys.path.append(SPACE['CODE_FN'])
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

from datasets import disable_caching
disable_caching()

SPACE['MODEL_ENDPOINT'] = 'vTest'

# Part 1: AIData

In [ ]:
# df_tag.columns

from recfldtkn.base import assign_caseSplitTag_to_dsCase
from recfldtkn.base import apply_multiple_conditions
import numpy as np 


## Eval: Food Event

In [ ]:
from datasets import load_from_disk

######## based patients with before 24h food event ###########
AIDataName = 'EventFood2CGM_bf5min_WellDoc_v2_v0323'
path = os.path.join(SPACE['DATA_AIDATA'], AIDataName)
print(path)
dataset = load_from_disk(path)

In [ ]:
columns = dataset.column_names
columns_tag = [i for i in columns if '--' not in i]
df_tag = dataset.select_columns(columns_tag).to_pandas()


df_pid = dataset.select_columns(['PID', 'GenderGroup', 'DiseaseTypeGroup', 'AgeGroup']).to_pandas()
df_pid = df_pid.drop_duplicates()
df_pid = df_pid[df_pid['GenderGroup'].isin(['Gender.1', 'Gender.2'])].reset_index(drop=True)
df_pid = df_pid[df_pid['DiseaseTypeGroup'].isin(['DiseaseType.1.0', 'DiseaseType.2.0'])].reset_index(drop=True)
df_display = df_pid.groupby(['GenderGroup', 'DiseaseTypeGroup', 'AgeGroup']).count().reset_index()# .sample(5)
display(df_display)

pid_selected = list(set(df_pid['PID'].values))
print(len(pid_selected)) # 647

PID_with_food_full = list(set(dataset['PID']))
print(len(PID_with_food_full)) # 654

In [ ]:
len(df_tag)
6 / 28

In [ ]:
df_tag['Year'].value_counts().sort_index()

In [ ]:
Split_to_Selection_Food = {
    'eval_food_t1d_train': {
        'Rules': [
            # ['PID', 'in', pid_selected],
            # ['GenderGroup', '==', 'Gender.1'],
            ['DiseaseTypeGroup', '==', 'DiseaseType.1.0'],
            # ['AgeGroup', '==', '40-64'],
            ['Split', '==', 'Train'],
        ], 
        'Op': 'and',
    },

    'eval_food_t2d_train': {
        'Rules': [
            # ['PID', 'in', pid_selected],
            # ['GenderGroup', '==', 'Gender.1'],
            ['DiseaseTypeGroup', '==', 'DiseaseType.2.0'],
            # ['AgeGroup', '==', '40-64'],
            ['Split', '==', 'Train'],
        ], 
        'Op': 'and',
    },


    'eval_food_t1d_valid': {
        'Rules': [
            # ['PID', 'in', pid_selected],
            # ['GenderGroup', '==', 'Gender.2'],
            ['DiseaseTypeGroup', '==', 'DiseaseType.1.0'],
            # ['AgeGroup', '==', '40-64'],
            ['Split', '==', 'Valid'],
        ], 
        'Op': 'and',
    },

    'eval_food_t2d_valid': {
        'Rules': [
            # ['PID', 'in', pid_selected],
            # ['GenderGroup', '==', 'Gender.2'],
            ['DiseaseTypeGroup', '==', 'DiseaseType.2.0'],
            # ['AgeGroup', '==', '40-64'],
            ['Split', '==', 'Valid'],
        ], 
        'Op': 'and',
    },

    'eval_food_t1d_test': {
        'Rules': [
            # ['PID', 'in', pid_selected],
            # ['GenderGroup', '==', 'Gender.2'],
            ['DiseaseTypeGroup', '==', 'DiseaseType.1.0'],
            # ['AgeGroup', '==', '40-64'],
            ['Split', '==', 'Test'],
        ], 
        'Op': 'and',
    },

    'eval_food_t2d_test': {
        'Rules': [
            # ['PID', 'in', pid_selected],
            # ['GenderGroup', '==', 'Gender.2'],
            ['DiseaseTypeGroup', '==', 'DiseaseType.2.0'],
            # ['AgeGroup', '==', '40-64'],
            ['Split', '==', 'Test'],
        ], 
        'Op': 'and',
    },
}

split_to_dataset = {}
for split_name, Selection in Split_to_Selection_Food.items():
    # split_to_dataset[split_name] = dataset.filter(lambda x: apply_multiple_conditions(x, split_config['Rules'], split_config['Op']))
    Rules = Selection['Rules']
    Op = Selection['Op']

    index = apply_multiple_conditions(df_tag, Rules, Op)
    indices = np.where(index == 1)[0]
    # len(indices)
    dataset_selected = dataset.select(indices)
    split_to_dataset[split_name] = dataset_selected

split_to_dataset_food = split_to_dataset
print(split_to_dataset_food)

# Train: T1D & T2D in 123 Before 2023

In [ ]:
# Oneday: 288, 24pd. 1/12
from datasets import load_from_disk

# 24 / 288
AIDataName = 'CGM_32h_24pd_WellDoc_v2_v0323' # CGM, 32h, 24 data per day. 

path = os.path.join(SPACE['DATA_AIDATA'], AIDataName)
print(path)
dataset = load_from_disk(path)
# dataset

config = dataset.info.__dict__['config_name']# .features['cf'].feature.vocab
print([i for i in config])
CF_to_CFvocab = config['CF_to_CFvocab']
print([i for i in CF_to_CFvocab])

CF_to_CFArgs = config['CaseSettingInfo']['Case_Args_Settings']['CF_to_CFArgs']
print([i for i in CF_to_CFArgs])

TriggerCaseBaseName = config['TriggerCaseBaseName']
TriggerCaseBaseArgs = config['TriggerCaseBaseName_to_TriggerCaseBaseArgs'][TriggerCaseBaseName]
TriggerName = TriggerCaseBaseArgs['Trigger']['TriggerName']
TriggerName
# print(TriggerCaseBaseArgs)

In [ ]:
columns = dataset.column_names
columns_tag = [i for i in columns if '--' not in i]
df_tag = dataset.select_columns(columns_tag).to_pandas()

In [ ]:
Split_to_Selection = {
    'Train': {
        'Rules': [
            ['PID', 'not in', PID_with_food_full],
            # ['Age', '>=', 40],
            ['Cohort', 'in', ['1', '2', '3']], # <--- add Cohort column
            ['Year', 'in', [2020, 2021, 2022, 2023]], # <--- add Year column
            ['GenderGroup', 'in', ['Gender.1', 'Gender.2']], 
            ['DiseaseTypeGroup', 'in', ['DiseaseType.1.0', 'DiseaseType.2.0']],
            ['ObsDT', '<',  '2022-07-01'], 
            ['ObsDT', '>=', '2021-01-01'],
        ], 
        'Op': 'and',
    },


    'Val_T1D': {
        'Rules': [
            ['PID', 'not in', PID_with_food_full],
            # ['Age', '>=', 40],
            ['Cohort', 'in', ['1', '2', '3']], # <--- add Cohort column
            ['Year', 'in', [2020, 2021, 2022, 2023]], # <--- add Year column
            ['ObsDT', '<',  '2023-01-01'], 
            ['ObsDT', '>=', '2022-07-01'],
            ['GenderGroup', 'in', ['Gender.1', 'Gender.2']], 
            ['DiseaseTypeGroup', 'in', ['DiseaseType.1.0']],
        ], 
        'Op': 'and',
    },

    'Val_T2D': {
        'Rules': [
            ['PID', 'not in', PID_with_food_full],
            # ['Age', '>=', 40],
            ['Cohort', 'in', ['1', '2', '3']], # <--- add Cohort column
            ['Year', 'in', [2020, 2021, 2022, 2023]], # <--- add Year column
            ['ObsDT', '<',  '2023-01-01'], 
            ['ObsDT', '>=', '2022-07-01'],
            ['GenderGroup', 'in', ['Gender.1', 'Gender.2']], 
            ['DiseaseTypeGroup', 'in', ['DiseaseType.2.0']],
        ], 
        'Op': 'and',
    },

    
    'Test_T1D': {
        'Rules': [
            ['PID', 'not in', PID_with_food_full],
            # ['Age', '>=', 40],
            ['Cohort', 'in', ['1', '2', '3']], # <--- add Cohort column
            ['Year', 'in', [2020, 2021, 2022, 2023]], # <--- add Year column
            ['ObsDT', '>=', '2023-01-01'], 
            ['ObsDT', '<',  '2024-01-01'],
            ['GenderGroup', 'in', ['Gender.1', 'Gender.2']], 
            ['DiseaseTypeGroup', 'in', ['DiseaseType.1.0']],
        ], 
        'Op': 'and',
    },
    'Test_T2D': {
        'Rules': [
            ['PID', 'not in', PID_with_food_full],
            # ['Age', '>=', 40],
            ['Cohort', 'in', ['1', '2', '3']], # <--- add Cohort column
            ['Year', 'in', [2020, 2021, 2022, 2023]], # <--- add Year column
            ['ObsDT', '>=', '2023-01-01'], 
            ['ObsDT', '<',  '2024-01-01'],
            ['GenderGroup', 'in', ['Gender.1', 'Gender.2']], 
            ['DiseaseTypeGroup', 'in', ['DiseaseType.2.0']],
        ], 
        'Op': 'and',
    }
}

In [ ]:
split_to_dataset = {}
for split_name, Selection in Split_to_Selection.items():
    # split_to_dataset[split_name] = dataset.filter(lambda x: apply_multiple_conditions(x, split_config['Rules'], split_config['Op']))
    Rules = Selection['Rules']
    Op = Selection['Op']

    index = apply_multiple_conditions(df_tag, Rules, Op)
    indices = np.where(index == 1)[0]
    # len(indices)
    dataset_selected = dataset.select(indices)
    split_to_dataset[split_name] = dataset_selected

split_to_dataset

## Data Description

In [ ]:
for k, v in split_to_dataset_food.items():
    split_to_dataset[k] = v 

split_to_dataset

In [ ]:
columns = [
    'DiseaseTypeGroup', 'GenderGroup', 'AgeGroup'
]

for SplitName in Split_to_Selection.keys():
    print(f'\n========== {SplitName} ==========' )

    ds = split_to_dataset[SplitName]
    columns_tag = ds.column_names
    columns_tag = [i for i in columns_tag if '--' not in i]
    df_tag = ds.select_columns(columns_tag).to_pandas()

    v = df_tag[columns].value_counts().sort_index().reset_index()
    display(v)

    v = df_tag[['PID'] + columns].drop_duplicates()[columns].value_counts().sort_index().reset_index()
    display(v)

In [ ]:
Name_to_Data = {}
for split, dataset in split_to_dataset.items():
    Name_to_Data[split] = {'ds_case': dataset}
Name_to_Data

## Set Transform

In [ ]:
OneEntryArgs = {
     # ----------------- Input Part -----------------
    'Input_Part': {
        'EntryInputMethod': 'Mto1Period_MultiTknInStepNoWgt',
        'CF_list': [
            'cf.TargetCGM_Bf24H',
            'cf.TargetCGM_Af2H',
            # 'cf.TargetCGM_Af2Hto8H',
        ],
        'TargetField': 'TargetCGM',
        'BeforePeriods': ['Bf24H'],
        'AfterPeriods': ['Af2H'],
        'InferenceMode': False, # 'WithFutureEvent' #  # 'NoFutureEvent', 'WithFutureEvent', 
    }, 

    # ----------------- Output Part -----------------
    # 'Output_Part': {
    #     'EntryOutputMethod': 'CausalLM',
    #     'set_transform': True,
    #     'num_proc': 4, 
    # },

    # 'Output_Part': {
    #     'EntryOutputMethod': 'MaskedLM',
    #     'MaskingRate': 0.15,
    #     'set_transform': True,
    #     'num_proc': 4, 
    # },

    'Output_Part': {
        'EntryOutputMethod': 'SupervisedFT',
        'AfStepNum': 24, # 12, # assert AfterPeriods Af2H,so 12 * 2 = 24
        'set_transform': True,
        'num_proc': 4, 
    },
}

from recfldtkn.aidata_base.entry import EntryAIData_Builder

entry = EntryAIData_Builder(TriggerName = TriggerName, 
                            OneEntryArgs = OneEntryArgs, 
                            SPACE = SPACE)

In [ ]:
Name_to_Data = entry.setup_EntryFn_to_NameToData(Name_to_Data, CF_to_CFvocab, OneEntryArgs)
Name_to_Data

## View A Batch

In [ ]:
Data = Name_to_Data['Train']
# Data

Data

In [ ]:
ds_tfm = Data['ds_tfm']
ds_tfm

In [ ]:
batch_size = 4
batch = ds_tfm[:batch_size]
batch

In [ ]:
for k, v in batch.items():
    print(k, v.shape)

# 4: batch size 
# 313: 288 (24h) +  1 (obspoint) + 24 (2h) # sequence length


# input_ids torch.Size([4, 313])
# labels torch.Size([4, 313])

In [ ]:
32 * 12 + 1

In [ ]:
batch['input_ids'][0] # .shape # 

In [ ]:
# softmax ignore -100
batch['labels'][0]

In [ ]:
# batch['Diet--event_indicators'][0]# .sum(axis = 1)

In [ ]:
input_ids = batch['input_ids']
input_ids.shape

In [ ]:
input_ids

In [ ]:
input_ids[2, :] # 313 = 288 (24h) +  1 (obspoint) + 24 (2h)

In [ ]:
labels = batch['labels']
labels.shape